In [1]:
# 1. Load code/data
!git clone https://github.com/joms-hub/tagalog-fake-news-detection.git
import os
os.chdir('/kaggle/working/tagalog-fake-news-detection')

# 2. Install packages (if needed)
!pip install transformers datasets evaluate huggingface_hub accelerate torch

Cloning into 'tagalog-fake-news-detection'...
remote: Enumerating objects: 207, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 207 (delta 106), reused 137 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (207/207), 4.87 MiB | 16.55 MiB/s, done.
Resolving deltas: 100% (106/106), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
import json

def patch_feature_type(path):
    with open(path, 'r') as f:
        data = json.load(f)
    # Recursively replace "List" with "Sequence"
    def replace_list(obj):
        if isinstance(obj, dict):
            return {k: replace_list(v) for k, v in obj.items()}
        elif obj == "List":
            return "Sequence"
        elif isinstance(obj, list):
            return [replace_list(x) for x in obj]
        else:
            return obj
    data['features'] = replace_list(data['features'])
    with open(path, 'w') as f:
        json.dump(data, f, indent=2)

# Patch all three dataset info files
patch_feature_type("/kaggle/working/tagalog-fake-news-detection/tokenized/TinyBERT_train/dataset_info.json")
patch_feature_type("/kaggle/working/tagalog-fake-news-detection/tokenized/TinyBERT_val/dataset_info.json")
patch_feature_type("/kaggle/working/tagalog-fake-news-detection/tokenized/TinyBERT_test/dataset_info.json")

In [2]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 24.3 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 3.6.0
    Uninstalling datasets-3.6.0:
      Successfully uninstalled datasets-3.6.0


In [3]:
# 3. Load preprocessed datasets
from datasets import load_from_disk
train_dataset = load_from_disk("/kaggle/working/tagalog-fake-news-detection/tokenized/TinyBERT_train")
val_dataset = load_from_disk("/kaggle/working/tagalog-fake-news-detection/tokenized/TinyBERT_val")
test_dataset  = load_from_disk("/kaggle/working/tagalog-fake-news-detection/tokenized/TinyBERT_test")


In [4]:
print(train_dataset, val_dataset, test_dataset)

Dataset({
    features: ['label', 'article', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2244
}) Dataset({
    features: ['label', 'article', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 481
}) Dataset({
    features: ['label', 'article', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 481
})


In [5]:

# 4. Set up model
from transformers import BertForSequenceClassification
model_name = "huawei-noah/TinyBERT_General_4L_312D"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

2025-09-11 15:18:16.374797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757603896.565924      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757603896.618411      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

In [6]:

# 5. Training with early stopping
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from evaluate import load
import numpy as np

os.environ["WANDB_DISABLED"] = "true"  # Disable wandb logging

def compute_metrics(eval_pred):
    f1_metric = load("f1")
    acc_metric = load("accuracy")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    f1_result = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    acc_result = acc_metric.compute(predictions=predictions, references=labels)
    return {
        "f1": f1_result["f1"],
        "accuracy": acc_result["accuracy"]
    }

# Adjust batch sizes if you hit RAM limits (Kaggle RAM is 16GB)
training_args = TrainingArguments(
    output_dir='/kaggle/working/tagalog-fake-news-detection/results',
    num_train_epochs=50,
    per_device_train_batch_size=32,   # Reduce from 32 if you get OOM errors
    per_device_eval_batch_size=32,    # Same here
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='/kaggle/working/tagalog-fake-news-detection/logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=None
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,0.455914,0.830921,0.831601
2,0.546000,0.408501,0.840923,0.841996
3,0.393100,0.381298,0.853431,0.854470
4,0.393100,0.366253,0.860216,0.860707
5,0.335000,0.328972,0.881321,0.881497
6,0.286700,0.325877,0.881451,0.881497
7,0.286700,0.302058,0.889812,0.889813
8,0.249100,0.302073,0.900117,0.900208
9,0.239700,0.351648,0.887310,0.887734
10,0.165900,0.284691,0.906422,0.906445


TrainOutput(global_step=923, training_loss=0.26398589822866286, metrics={'train_runtime': 306.7192, 'train_samples_per_second': 365.807, 'train_steps_per_second': 11.574, 'total_flos': 418297197846528.0, 'train_loss': 0.26398589822866286, 'epoch': 13.0})

In [7]:
# 6. Logging and Saving Outputs
import json
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

label_names = ['Real', 'Fake']  # Update if your labels are different

predictions = trainer.predict(test_dataset)

output_dir = "/kaggle/working/tagalog-fake-news-detection/outputs"
os.makedirs(output_dir, exist_ok=True)

metrics = predictions.metrics
metrics_file = os.path.join(output_dir, 'tinybert_metrics.json')
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"Saved metrics to {metrics_file}")

preds = np.argmax(predictions.predictions, axis=1)
cm = confusion_matrix(predictions.label_ids, preds)
cm_df = pd.DataFrame(cm, index=label_names, columns=[f'Predicted {n}' for n in label_names])
print("\nConfusion Matrix:")
print(cm_df)

cm_file = os.path.join(output_dir, 'tinybert_cm.csv')
cm_df.to_csv(cm_file)
print(f"Saved confusion matrix to {cm_file}")

results_df = pd.DataFrame({
    'true_label': predictions.label_ids,
    'predicted_label': preds
})
results_file = os.path.join(output_dir, 'tinybert_predictions.csv')
results_df.to_csv(results_file, index=False)
print(f"Saved raw predictions to {results_file}")

report = classification_report(predictions.label_ids, preds, target_names=label_names, output_dict=True)
report_file = os.path.join(output_dir, 'tinybert_classification_report.json')
with open(report_file, 'w') as f:
    json.dump(report, f, indent=2)
print("\nClassification Report:")
print(pd.DataFrame(report).transpose())
print(f"Saved classification report to {report_file}")


Saved metrics to /kaggle/working/tagalog-fake-news-detection/outputs/tinybert_metrics.json

Confusion Matrix:
      Predicted Real  Predicted Fake
Real             224              17
Fake              46             194
Saved confusion matrix to /kaggle/working/tagalog-fake-news-detection/outputs/tinybert_cm.csv
Saved raw predictions to /kaggle/working/tagalog-fake-news-detection/outputs/tinybert_predictions.csv

Classification Report:
              precision    recall  f1-score     support
Real           0.829630  0.929461  0.876712  241.000000
Fake           0.919431  0.808333  0.860310  240.000000
accuracy       0.869023  0.869023  0.869023    0.869023
macro avg      0.874530  0.868897  0.868511  481.000000
weighted avg   0.874437  0.869023  0.868528  481.000000
Saved classification report to /kaggle/working/tagalog-fake-news-detection/outputs/tinybert_classification_report.json


In [8]:

# 7. Push trained model to HuggingFace
from huggingface_hub import login
login("")  # Replace with your token
trainer.push_to_hub("tinybert-fnf")

Uploading...:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jcunado/results/commit/d36ee72e0bb4babea905c20b570c7c4e7abf5635', commit_message='tinybert-fnf', commit_description='', oid='d36ee72e0bb4babea905c20b570c7c4e7abf5635', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jcunado/results', endpoint='https://huggingface.co', repo_type='model', repo_id='jcunado/results'), pr_revision=None, pr_num=None)

In [9]:
%cd /kaggle/working/tagalog-fake-news-detection/
!git config --global user.name "joms-hub"
!git config --global user.email "21101932@usc.edu.ph"
!git remote set-url origin https://@github.com/joms-hub/tagalog-fake-news-detection.git

/kaggle/working


In [10]:
!git pull
!git add logs outputs
!git commit -m "new tinybert logs and outputs"
!git push

Already up to date.
[main 1e30282] new tinybert logs and outputs
 5 files changed, 520 insertions(+)
 create mode 100644 logs/events.out.tfevents.1757604046.5d3314e8e828.36.0
 create mode 100644 outputs/tinybert_classification_report.json
 create mode 100644 outputs/tinybert_cm.csv
 create mode 100644 outputs/tinybert_metrics.json
 create mode 100644 outputs/tinybert_predictions.csv
Enumerating objects: 12, done.
Counting objects: 100% (12/12), done.
Delta compression using up to 4 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 5.63 KiB | 5.63 MiB/s, done.
Total 9 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
remote: Bypassed rule violations for refs/heads/main:
remote: 
remote: - Changes must be made through a pull request.
remote: 
To https://github.com/joms-hub/tagalog-fake-news-detection.git
   fd487b7..1e30282  main -> main
